In [1]:
using Pkg;  Pkg.add(Pkg.PackageSpec(;name="Plots", version="0.22.1"))

  Updating registry at `~/.julia/registries/General`


┌ Warning: Some registries failed to update:
│     — `~/.julia/registries/General` — registry dirty
└ @ Pkg.Types /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Pkg/src/Types.jl:1269


 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]


In [2]:
using Pkg;
pkgs = Pkg.installed();
pkgs["Plots"]

v"0.22.1"

In [1]:
#Include
using LightGraphs, SparseArrays, SimpleWeightedGraphs
using Statistics, BenchmarkTools, LinearAlgebra, ProgressMeter
using Base.Threads, PhyloNetworks, StatsBase, Distributions
using Base.GC, JLD2, FileIO, CSV, DataFrames
using Random, NPZ, GraphRecipes, Plots

In [97]:
include("TreeRep.jl")
include("ConstructTree.jl")
include("LevelTree.jl")
include("NJ.jl")
include("Utilities.jl")
include("Visualize.jl")
include("TreeOpt.jl")
include("SparseRep.jl")
include("Bartal.jl")

Main.Bartal

In [113]:
plotly()

Plots.PlotlyBackend()

In [5]:
utilities.tm()

Number of threads = 8
  0.508864 seconds
  0.073148 seconds (539 allocations: 35.440 KiB)
  0.378764 seconds
  0.052161 seconds (1 allocation: 48 bytes)


In [18]:
G = BinaryTree(5)
D = LightGraphs.floyd_warshall_shortest_paths(G).dists;

In [19]:
utilities.calc_delta(D)

0

In [ ]:
Visualize.visualize(G)
plot!(legend=:false)

In [40]:
font = Plots.font("Helvetica", 15)
font2 = Plots.font("Helvetica", 9)
myfonts = Dict(:guidefont=>font, :xtickfont=>font2, :ytickfont=>font2, :legendfont=>font2)

Dict{Symbol,Plots.Font} with 4 entries:
  :legendfont => Font("Helvetica", 9, :hcenter, :vcenter, 0.0, RGB{N0f8}(0.0,0.…
  :guidefont  => Font("Helvetica", 15, :hcenter, :vcenter, 0.0, RGB{N0f8}(0.0,0…
  :xtickfont  => Font("Helvetica", 9, :hcenter, :vcenter, 0.0, RGB{N0f8}(0.0,0.…
  :ytickfont  => Font("Helvetica", 9, :hcenter, :vcenter, 0.0, RGB{N0f8}(0.0,0.…

In [134]:
@elapsed G2,W2 = TreeRep.metric_to_structure(D,undef,undef)

flush(stdout)
B = W2[1:nv(G2),1:nv(G2)];
B = sparse(B);
B = (B .> 0) .* B;

(nv(G), ne(G)) = (14, 13)


In [164]:
Visualize.visualize(R,W = wbfs, labels = [text("Dog", :top, 20),text("Bear", :left, 20),
        text("Racoon", :top, 20), text("Weasel", :left, 20),text("Seal", :bottom, 20),
        text("Sea Lion", :top, 20),
        text("Cat", :left, 20),
        text("Monkey", :right, 20),
        "","","","","","",""])

In [165]:
plot!(legend=:false,axis=:false)

In [137]:
Visualize.visualize(T,W = W4, labels = ["dog","bear","racoon", "weasel","seal","sea lion","cat","monkey     ",
        "","","","","","",""])

UndefVarError: UndefVarError: W4 not defined

In [12]:
plot!(legend=:false)

In [ ]:
Ph = CSV.read("./../../../julia-1.1.0/bin/phylo-embed")
P = Matrix(Ph[1:end,[2,3]])

In [ ]:
scatter(P[:,1],P[:,2])

In [ ]:
for e in edges(g)
    i = e.dst
    j = e.src
    
    plot!([(P[i,1],P[i,2]),(P[j,1],P[j,2])],color=:black)
end

In [ ]:
plot!(legend=:false)

# Temp

In [68]:
P = zeros(8,2)

8×2 Array{Float64,2}:
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

In [69]:
P[1,:] = [-0.6015,  -0.7989]
P[2,:] = [0.8426,  0.5386]
P[3,:] = [-0.8188,  0.5740]
P[4,:] = [0.7942, -0.6047]
P[5,:] = [-0.9950, -0.0819]
P[6,:] = [-0.1758,  0.9827]
P[7,:] = [0.0065, -0.9208]
P[8,:] = [-0.0761, -0.1009]

2-element Array{Float64,1}:
 -0.0761
 -0.1009

In [70]:
P

8×2 Array{Float64,2}:
 -0.6015  -0.7989
  0.8426   0.5386
 -0.8188   0.574 
  0.7942  -0.6047
 -0.995   -0.0819
 -0.1758   0.9827
  0.0065  -0.9208
 -0.0761  -0.1009

In [154]:
scatter(P[:,1],P[:,2], series_annotations = [text("Dog", :right, 20),text("Bear", :right, 20),
        text("Racoon", :right, 20), text("Weasel", :right, 20),text("Seal", :right, 20),
        text("Sea Lion", :right, 20),
        text("Cat", :right, 20),
        text("Monkey", :right, 20)],
legend=:false, ms = 1; myfonts..., axis=:false)

In [78]:
P2 = zeros(8,2)

8×2 Array{Float64,2}:
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

In [86]:
T = npzread("./sarich.final_coordinates0.npy")
P2[1,:] = T[2:end]/(1+T[1])

2-element Array{Float64,1}:
 -0.0001866253716251444
 -0.0005304081715788266

In [155]:
scatter(P2[:,1],P2[:,2], series_annotations = [text("Dog", :right, 20),text("Bear", :right, 20),
        text("Racoon", :top, 20), text("Weasel", :bottom, 20),text("Seal", :right, 20),
        text("Sea Lion", :right, 20),
        text("Cat", :right, 20),
        text("Monkey", :right, 20)],
legend=:false, axis=:false)

# Bartal

In [ ]:
g = BinaryTree(4)
D = floyd_warshall_shortest_paths(g).dists
n = nv(g)

In [ ]:
D7 = zeros(n,n)
p2 = Progress(2)
for i = 1:2
    R = Bartal.bartal(g,collect(1:n),D)
    D6 = utilities.parallel_dp_shortest_paths(R[1],R[4])
    p = R[2]
    IndexToIdx = copy(R[2])
    for i = 1:length(p)
        IndexToIdx[p[i]] = i
    end
    D2p = zeros(n,n)
    for i = 1:length(p)
        for j = 1:i-1
            D2p[i,j] = D6[IndexToIdx[i],IndexToIdx[j]]
            D2p[j,i] = D2p[i,j]
        end
    end
    D7 = (D7*(i-1) + D2p)/i
    update!(p2,i)
    flush(stdout)
end
α = tr(D7[1:n,1:n]'*D)/tr(D7[1:n,1:n]'*D7[1:n,1:n])
@show(utilities.avg_distortion(D7*α,D))
@show(utilities.MAP(D7,g))

In [ ]:
D7

In [ ]:
α = tr(D7[1:n,1:n]'*D)/tr(D7[1:n,1:n]'*D7[1:n,1:n])
@show(utilities.avg_distortion(D7*α,D))
@show(utilities.MAP(D7,g))

In [ ]:
G = BinaryTree(4)
D = LightGraphs.floyd_warshall_shortest_paths(G).dists

In [ ]:
R = ldrd(G,D,diameter(G,adjacency_matrix(G))/2)

# Scotland Data

In [ ]:
D6 = utilities.parallel_dp_shortest_paths(R[1],R[4])

In [ ]:
F = CSV.read("2lev-xc.txt");

In [ ]:
X = Matrix(F[:,[1,2,3,5,6]])
X[:,3] = 1000*X[:,3]
X[:,5] = 100000*X[:,5];

In [ ]:
c = convert(Array{Int64,1},X[:,end]);

In [ ]:
D = zeros(3434,3434)
for i = 1:size(D)[1]
    for j = 1:i-1
        D[i,j] = sqrt(sum((X[i,:]-X[j,:]).^2))
    end
end

D = D + D';

In [ ]:
D = D ./ maximum(D)

In [ ]:
@elapsed G2,W2 = TreeRep.metric_to_structure(D,undef,undef);

In [ ]:
m = maximum(c)
c = vcat(c, (m+1)*convert(Array{Int64,1},ones(nv(G2)-length(c))));

In [ ]:
B = W2[1:nv(G2),1:nv(G2)];
B = sparse(B);
B = (B .> 0) .* B;
    
n = size(D)[1]
D2 = utilities.parallel_dp_shortest_paths(G2, B, false);
for i = 1:n
    D2[i,i] = 0
end
    
@show(utilities.avg_distortion(D2,D))

In [ ]:
Dist = abs.((D2 - D)./D)
for i = 1:size(Dist)[1]
    Dist[i,i] = 0
end

sum(D.==0)

# Random tree test

In [166]:
N = 7

T1 = zeros(N)
T2 = zeros(N)
map1 = zeros(N)
map2 = zeros(N)
dist1 = zeros(N)
dist2 = zeros(N)
nvs1 = zeros(N)
nvs2 = zeros(N)
nvs = zeros(N)

7-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [167]:
for i = 1:7
    g = utilities.block(LightGraphs.SimpleGraphs.DoubleBinaryTree(i),10)
    n = nv(g)
    g = LightGraphs.bfs_tree(g,rand(1:n))
    G = SimpleGraph(n)
    for e in edges(g)
        add_edge!(G,e)
    end
    @show(G)
    W = rand(n,n)
    W = W+W'
    W = adjacency_matrix(G) .* W
    D = utilities.parallel_dp_shortest_paths(G,W);
    T1[i] = @elapsed G2,W2 = TreeRep.metric_to_structure(D,undef,undef);
    
    nvs[i] = nv(G)
    

    B = W2[1:nv(G2),1:nv(G2)];
    B = sparse(B);
    B = (B .> 0) .* B;
    
    D2 = utilities.parallel_dp_shortest_paths(G2, B, false);
    for i = 1:n
        D2[i,i] = 0
    end
    
    nvs1[i] = nv(G2)
    
    map1[i] = utilities.MAP(D2[1:n,1:n],G)/utilities.MAP(D,G)
    dist1[i] = utilities.avg_distortion(D2[1:n,1:n],D)
    
    
    T2[i] = @elapsed R =  NJ.nj!(copy(convert(Matrix{Float64},D)));
    
    g2 = SimpleGraph(R.numNodes)
    w = spzeros(R.numNodes,R.numNodes)
    for i = 1:R.numEdges
        src = R.edge[i].node[1].number
        dst = R.edge[i].node[2].number
        add_edge!(g2,src,dst)
        w[src,dst] = R.edge[i].length
        w[dst,src] = w[src,dst]
    end
    
    nvs2[i] = nv(g2)
    
    D5 = utilities.parallel_dp_shortest_paths(g2, w)


    dist2[i] = utilities.avg_distortion(D5[1:n,1:n],D);
    map2[i] = utilities.MAP(D5[1:n,1:n],G)/utilities.MAP(D,G)
    
end

@show(T1,T2,map1,map2,dist1,dist2,nvs,nvs1,nvs2)

G = {11, 10} undirected simple Int64 graph
(nv(G), ne(G)) = (11, 10)
G = {40, 39} undirected simple Int64 graph
(nv(G), ne(G)) = 

┌ Info: 2 branches had negative lengths, reset to 0
└ @ Main.NJ /home/rishi/ICLR/Experiments/Gene_Expression/NJ.jl:141


(40, 39)
G = {89, 88} undirected simple Int64 graph
(nv(G), ne(G)) = (89, 88)
G = {191, 190} undirected simple Int64 graph
(nv(G), ne(G)) = (191, 190)


┌ Info: 16 branches had negative lengths, reset to 0
└ @ Main.NJ /home/rishi/ICLR/Experiments/Gene_Expression/NJ.jl:141
┌ Info: 29 branches had negative lengths, reset to 0
└ @ Main.NJ /home/rishi/ICLR/Experiments/Gene_Expression/NJ.jl:141


G = {362, 361} undirected simple Int64 graph
(nv(G), ne(G)) = 

...

Progress:  98%|████████████████████████████████████████▏|  ETA: 0:00:00

(nv(G), ne(G)) = (1611, 1610)


┌ Info: 504 branches had negative lengths, reset to 0
└ @ Main.NJ /home/rishi/ICLR/Experiments/Gene_Expression/NJ.jl:141
Progress: 100%|████████████████████████████████████████▊|  ETA: 0:00:00

T1 = [0.0526076, 0.229354, 0.00170722, 0.00391346, 0.0196504, 0.0802664, 0.123146]
T2 = [0.0835829, 0.00159493, 0.00667716, 0.0359934, 0.181397, 1.66127, 15.119]
map1 = [1.0, 1.0, 1.0, 1.0, 1.0, 0.997445, 0.999308]
map2 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
dist1 = [3.40716e-17, 7.6335e-17, 6.22912e-17, 8.04225e-17, 7.20925e-17, 7.32211e-5, 8.95329e-5]
dist2 = [1.88391e-16, 1.97578e-16, 2.89881e-16, 4.90095e-16, 7.07757e-16, 1.53169e-15, 1.99219e-15]
nvs = [11.0, 40.0, 89.0, 191.0, 362.0, 817.0, 1611.0]
nvs1 = [11.0, 40.0, 89.0, 191.0, 362.0, 817.0, 1611.0]
nvs2 = [20.0, 78.0, 176.0, 380.0, 722.0, 1632.0, 3220.0]


7-element Array{Float64,1}:
   20.0
   78.0
  176.0
  380.0
  722.0
 1632.0
 3220.0

In [168]:
plot(nvs,T1,label="TreeRep",lc = :blue, linewidth = 2, xlabel = "Nodes", ylabel = "Time taken (Seconds)"; myfonts...)
scatter!(nvs,T1,label="TreeRep",xlabel = "Nodes", ylabel = "Time taken (Seconds)", mc = :blue, shape=:circle; myfonts...)
plot!(nvs,T2,label="NJ", lc = :red, linewidth = 2; myfonts...)
scatter!(nvs,T2,label="NJ", mc = :red, shape =:xcross; myfonts...)

In [ ]:
plot(nvs,nvs1./nvs,label = "TreeRep", xlabel = "Nodes", ylabel = "Ratio of number of nodes in returned tree to original tree")
plot!(nvs, nvs2./nvs,label = "NJ")

# Zeisel

In [34]:
@load "zeisel-dists.jld2" A2

1-element Array{Symbol,1}:
 :A2

In [ ]:
@load "cite-dists.jld2" A2

In [35]:
D = A2
n = size(D)[1]
G = CompleteGraph(n)

{3005, 4513510} undirected simple Int64 graph

In [6]:
g = CompleteGraph(size(D)[1])
E = ne(g)

id1 = convert(Array{Int64,1},zeros(E))
id2 = convert(Array{Int64,1},zeros(E))
weight = zeros(E)

Ed = collect(edges(g))

for i = 1:E
    e = Ed[i]
    id1[i] = e.src
    id2[i] = e.dst
    weight[i] = D[e.src,e.dst]
end

df = DataFrame(id1 = id1, id2 = id2, weight = weight)
CSV.write("sarich.csv",  df, writeheader=true)

"sarich.csv"

In [8]:
filename = "sarich."
CSV.write(filename*"edges",CSV.read(filename*"csv"); delim=' ',writeheader=false)

"sarich.edges"

In [ ]:
open("zeisel-tree.edges", "w") do f
    for e in edges(G2)
        i = e.src - 1
        j = e.dst - 1
        w = W2[i+1,j+1]/100
        if w < 0
            w = 0
        end
        write(f, "$i $j $w\n")
    end
end

In [ ]:
using MAT

X = matread("./zeisel_data.mat")["zeisel_data"]
N = ((ones(4000)'*X.^2)'*ones(3005)')
A2 = N + N' - 2* X'*X
for i = 1:3005
    A2[i,i] = 0
end
A2 = A2.^(0.5)

In [ ]:
d,n = size(X)
A = zeros(n,n)
@showprogress for i = 1:n
    for j = 1:i-1
        A[i,j] = sum((D[:,i]-D[:,j]).^2)^(0.5)
    end
end

# Load Data Sets

In [92]:
D = [ 0  32  48  51  50  48  98 148;
32   0  26  34  29  33  84 136;
48  26   0  42  44  44  92 152;
51  34  42   0  44  38  86 142;
50  29  44  44   0  24  89 142;
48  33  44  38  24   0  90 142;
98  84  92  86  89  90   0 148;
148 136 152 142 142 142 148 0]

8×8 Array{Int64,2}:
   0   32   48   51   50   48   98  148
  32    0   26   34   29   33   84  136
  48   26    0   42   44   44   92  152
  51   34   42    0   44   38   86  142
  50   29   44   44    0   24   89  142
  48   33   44   38   24    0   90  142
  98   84   92   86   89   90    0  148
 148  136  152  142  142  142  148    0

In [ ]:
G,w = utilities.read_tree_withweights("./../hyperbolics-master/data/edges/bio-CE-GT.edges"," ")
n = nv(G)
E = ne(G)
@show((n,E));
@show(is_connected(G));

In [12]:
G = utilities.remove_loops(utilities.read_tree("./../hyperbolics-master/data/edges/wordnet.edges"," "))
n = nv(G)
E = ne(G)
@show((n,E));
@show(is_connected(G));

(n, E) = (74374, 75834)
is_connected(G) = true


In [5]:
C = connected_components(G)
idxmax = argmax(length.(C))

2

In [6]:
g,V = induced_subgraph(G, C[idxmax])
@show(is_connected(g));
n = nv(g)
E = ne(g)

print(n," ",E)

is_connected(g) = true
33695 180810

In [7]:
W = w[V,V];

UndefVarError: UndefVarError: w not defined

In [8]:
W = adjacency_matrix(g);

In [9]:
@time D = utilities.parallel_dp_shortest_paths(g,adjacency_matrix(g));

Progress: 100%|█████████████████████████████████████████|  ETA: 0:00:00

299.370357 seconds (5.22 M allocations: 130.667 GiB, 11.61% gc time)


In [10]:
maximum(D)

13

In [58]:
6.5/28

0.23214285714285715

In [ ]:
d = utilities.calc_delta_for_w(D,1)

Progress:   5%|██                                       |  ETA: 0:35:09

In [60]:
3/17

0.17647058823529413

In [ ]:
celegans = 1.5, 0.021949574631569782
diseasome = 2.5, 0.03335496540574415
phd = 6.5, 0.08464395557806965
yeast = 
grid worm = 


In [127]:
@elapsed G2,W2 = TreeRep.metric_to_structure(D,undef,undef)

flush(stdout)
B = W2[1:nv(G2),1:nv(G2)];
B = sparse(B);
B = (B .> 0) .* B;

(nv(G), ne(G)) = (14, 13)


0.000623699

In [101]:
D2 = utilities.parallel_dp_shortest_paths(G2, B, false);

In [10]:
Sys.free_memory()/2^(30)

82.0709114074707

In [ ]:
utilities.avg_distortion(D2[1:n,1:n],D)

In [ ]:
utilities.MAP(D[1:n,1:n],g)

In [ ]:
SparseRep.add_kEdges(G2,B,g,D,20)

In [ ]:
SparseRep.repeat_merge(D,g,n,20)

In [ ]:
graphplot(D2.*adjacency_matrix(G2))

# Ultra method

In [125]:
function compute_Ca(D,a=1)
    n = size(D)[1]
    ma = maximum(D[a,:])
    Ca = 2*ma .- (ones(n)*D[a,:]' + D[a,:]*ones(n)')
    
    return Ca
end

function compute_U(M)
    n = size(M)[1]
    G = CompleteGraph(n)
    r = LightGraphs.prim_mst(G,M)
    R = SimpleGraph(n)
    for e in r
        add_edge!(R,e)
    end
    
    A = LightGraphs.adjacency_matrix(R)
    Mp = A .* M
    
    nextroots = collect(n+2:3*n)
    W = spzeros(3*n,3*n)
    
    W,nextroots = recurse_U(R,collect(1:n),Mp,nextroots,W,n+1)
    
    T = SimpleGraph(3*n)
    I,J,V = findnz(W)
    for k = 1:length(I)
        add_edge!(T,I[k],J[k])
        if W[I[k],J[k]] == -1
            W[I[k],J[k]] = 0
        end
    end
    
    return T,W
end

function recurse_U(T,V,M,nextroots,W,r)
    if length(V) == 1
        W[r,V[1]] = -1
        W[V[1],r] = -1
        
        return W,nextroots
    end
    
    r1 = nextroots[1]
    r2 = nextroots[2]
    
    n = size(M)[1]
    I,J,U = findnz(M)
    m = argmax(U)
    i = I[m]
    j = J[m]
    
    W[r,r1] = U[m]/2
    W[r,r2] = U[m]/2
    W[r2,r] = U[m]/2
    W[r1,r] = U[m]/2
    
    Aij = M[i,j]
    
    rem_edge!(T,i,j)
    C = connected_components(T)
    
    T1,V1 = induced_subgraph(T,C[1])
    T2,V2 = induced_subgraph(T,C[2])
    
    M1 = M[V1,V1]
    M2 = M[V2,V2]
    
    U1 = copy(V1)
    U2 = copy(V2)
    
    for k = 1:length(V1)
        V1[k] = V[V1[k]]
    end
    for k = 1:length(V2)
        V2[k] = V[V2[k]]
    end
    
    W,nextroots = recurse_U(T1,V1,M1,nextroots[3:end],W,r1)
    W,nextroots = recurse_U(T2,V2,M2,nextroots,W,r2)
   
    return W,nextroots
end

function eps(D)
    n = size(D)[1]
    Ca = compute_Ca(D)
    M = D + Ca
    T,W = compute_U(M)
    C = connected_components(T)
    T1,_ = induced_subgraph(T,C[argmax(length.(C))])
    U = LightGraphs.Parallel.floyd_warshall_shortest_paths(T1,W).dists
    AD = U[1:n,1:n] - Ca
    
    @time R = NJ.nj!(copy(convert(Matrix{Float64},AD)))
    g2 = SimpleGraph(R.numNodes)
    w = spzeros(R.numNodes,R.numNodes)
    for i = 1:R.numEdges
        src = R.edge[i].node[1].number
        dst = R.edge[i].node[2].number
        add_edge!(g2,src,dst)
        w[src,dst] = R.edge[i].length
        w[dst,src] = w[src,dst]
    end
    
    return g2,w
end
    

eps (generic function with 1 method)

In [135]:
g = BinaryTree(3)
Dg = LightGraphs.floyd_warshall_shortest_paths(g).dists

7×7 Array{Int64,2}:
 0  1  1  2  2  2  2
 1  0  2  1  1  3  3
 1  2  0  3  3  1  1
 2  1  3  0  2  4  4
 2  1  3  2  0  4  4
 2  3  1  4  4  0  2
 2  3  1  4  4  2  0

In [138]:
r = LightGraphs.prim_mst(g,Dg)
R = SimpleGraph(7)
for e in r
    println(e)
    add_edge!(R,e)
end

Edge 1 => 2
Edge 1 => 3
Edge 2 => 4
Edge 2 => 5
Edge 3 => 6
Edge 3 => 7


In [139]:
T,W = compute_U(Dg)

({21, 19} undirected simple Int64 graph, 
  [11,  1]  =  0.0
  [19,  2]  =  0.0
  [15,  3]  =  0.0
  [18,  4]  =  0.0
  [20,  5]  =  0.0
  [14,  6]  =  0.0
  [16,  7]  =  0.0
  [9 ,  8]  =  0.5
  [10,  8]  =  0.5
  [8 ,  9]  =  0.5
  [11,  9]  =  0.5
  [12,  9]  =  0.5
  ⋮
  [13, 15]  =  0.5
  [7 , 16]  =  0.0
  [13, 16]  =  0.5
  [10, 17]  =  0.5
  [19, 17]  =  0.5
  [20, 17]  =  0.5
  [4 , 18]  =  0.0
  [10, 18]  =  0.5
  [2 , 19]  =  0.0
  [17, 19]  =  0.5
  [5 , 20]  =  0.0
  [17, 20]  =  0.5)

In [140]:
C = connected_components(T)

2-element Array{Array{Int64,1},1}:
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
 [21]                                                                   

In [141]:
T1,_ = induced_subgraph(T,C[1])

({20, 19} undirected simple Int64 graph, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

In [142]:
for e in edges(T1)
    println(e)
end

Edge 1 => 11
Edge 2 => 19
Edge 3 => 15
Edge 4 => 18
Edge 5 => 20
Edge 6 => 14
Edge 7 => 16
Edge 8 => 9
Edge 8 => 10
Edge 9 => 11
Edge 9 => 12
Edge 10 => 17
Edge 10 => 18
Edge 12 => 13
Edge 12 => 14
Edge 13 => 15
Edge 13 => 16
Edge 17 => 19
Edge 17 => 20


In [143]:
D2 = LightGraphs.floyd_warshall_shortest_paths(T1,W).dists

20×20 Array{Float64,2}:
 0.0  2.5  2.0  2.0  2.5  1.5  2.0  1.0  …  1.5  2.0  2.0  2.0  2.0  2.5  2.5
 2.5  0.0  3.5  1.5  1.0  3.0  3.5  1.5     3.0  3.5  3.5  0.5  1.5  0.0  1.0
 2.0  3.5  0.0  3.0  3.5  1.5  1.0  2.0     1.5  0.0  1.0  3.0  3.0  3.5  3.5
 2.0  1.5  3.0  0.0  1.5  2.5  3.0  1.0     2.5  3.0  3.0  1.0  0.0  1.5  1.5
 2.5  1.0  3.5  1.5  0.0  3.0  3.5  1.5     3.0  3.5  3.5  0.5  1.5  1.0  0.0
 1.5  3.0  1.5  2.5  3.0  0.0  1.5  1.5  …  0.0  1.5  1.5  2.5  2.5  3.0  3.0
 2.0  3.5  1.0  3.0  3.5  1.5  0.0  2.0     1.5  1.0  0.0  3.0  3.0  3.5  3.5
 1.0  1.5  2.0  1.0  1.5  1.5  2.0  0.0     1.5  2.0  2.0  1.0  1.0  1.5  1.5
 0.5  2.0  1.5  1.5  2.0  1.0  1.5  0.5     1.0  1.5  1.5  1.5  1.5  2.0  2.0
 1.5  1.0  2.5  0.5  1.0  2.0  2.5  0.5     2.0  2.5  2.5  0.5  0.5  1.0  1.0
 0.0  2.5  2.0  2.0  2.5  1.5  2.0  1.0  …  1.5  2.0  2.0  2.0  2.0  2.5  2.5
 1.0  2.5  1.0  2.0  2.5  0.5  1.0  1.0     0.5  1.0  1.0  2.0  2.0  2.5  2.5
 1.5  3.0  0.5  2.5  3.0  1.0  0.5  1.5 

In [145]:
n = 7

7

In [151]:
D2[1:n,1:n] .>= Dg

7×7 BitArray{2}:
  true  true  true   true   true  false   true
  true  true  true   true   true   true   true
  true  true  true   true   true   true   true
  true  true  true   true  false  false  false
  true  true  true  false   true  false  false
 false  true  true  false  false   true  false
  true  true  true  false  false  false   true

# Construct Tree

In [10]:
g = CompleteGraph(n)

{3005, 4513510} undirected simple Int64 graph

In [142]:
global λ = 2000
n = size(D)[1]
@time T,W4 = ConstructTree.basicConstructTree(collect(2:n),1,D)

  0.147845 seconds (163.83 k allocations: 8.106 MiB)


({14, 13} undirected simple Int64 graph, [0.0 32.0 … 0.0 0.0; 32.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [10]:
@time D4 = LightGraphs.floyd_warshall_shortest_paths(T,W4[1:nv(T),1:nv(T)]).dists;

  0.138508 seconds (152.84 k allocations: 7.182 MiB)


In [20]:
α = tr(D4[1:n,1:n]'*D)/tr(D4[1:n,1:n]'*D4[1:n,1:n])

0.42210600715879026

In [21]:
utilities.avg_distortion(D4[1:n,1:n]*α,D)

0.7121231446658032

In [ ]:
utilities.MAP(D4[1:n,1:n],g)

# Calculating for Facebook

In [ ]:
Z = CSV.read("rand-dim$sc-scale1.csv")
D = zeros(100,100)
for k = 1:50*99
    i = Z[k,1]
    j = Z[k,2]
    D[i,j] = Z[k,3]
    D[j,i] = Z[k,3]
end

In [ ]:
dist7 = zeros(10)

In [5]:
p = collect(1:n) #npzread("./../../../grid-worm-order.npy");

In [6]:
IndexToIdx = copy(p)
for i = 1:length(p)
    IndexToIdx[p[i]] = i
end

In [7]:
IndexToIdx;

In [8]:
D2 = npzread("./zeisel-knn10.npy")
D2p = copy(D2)
for i = 1:length(p)
    for j = 1:i-1
        D2p[i,j] = D2[IndexToIdx[i],IndexToIdx[j]]
        D2p[j,i] = D2p[i,j]
    end
end

In [9]:
α = tr(D2p[1:n,1:n]'*D)/tr(D2p[1:n,1:n]'*D2p[1:n,1:n])

8.22483850533057

In [10]:
utilities.avg_distortion(D2p*α,D)

0.2944394342975348

In [ ]:
utilities.MAP(D2p[1:n,1:n],g)

# Calculate Hyperbolic_models

In [109]:
dist8 = zeros(10)
dist9 = zeros(10)

10-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [110]:
n=100

100

In [158]:
i = 10
sc = 2^i

1024

In [159]:
Z = CSV.read("rand-dim$sc-scale1.csv")
D = zeros(100,100)
for k = 1:50*99
    ii = Z[k,1]
    j = Z[k,2]
    D[ii,j] = Z[k,3]
    D[j,ii] = Z[k,3]
end

In [160]:
D2 = npzread("./rand-dim$sc-scale1-l-1024npy.npy")
for i = 1:n
    D2[i,i] = 0
end

In [161]:
α = tr(D2[1:n,1:n]'*D)/tr(D2[1:n,1:n]'*D2[1:n,1:n])

0.38430030936803233

In [162]:
dist8[i] = utilities.avg_distortion(D2*α,D)

0.10490860563948282

In [151]:
utilities.MAP(D2[1:n,1:n],g)

1.000000000000027

In [ ]:
neighborhood(g,1,1)

# Level version algoriton

In [14]:
@time gT = utilities.remove_loops(LevelTree.build_level_graph(g,D,1))
DT = utilities.parallel_dp_shortest_paths(gT, adjacency_matrix(gT));

  0.638097 seconds (1.02 M allocations: 87.435 MiB)


In [15]:
utilities.MAP(DT,g)

0.2722030902680685

In [18]:
α = tr(DT[1:n,1:n]'*D)/tr(DT[1:n,1:n]'*DT[1:n,1:n])
utilities.avg_distortion(DT[1:n,1:n]*α,D)

0.16642753426157253

# Tree Rep

In [ ]:
global p2 = Progress(nv(g))
global jj = 0

In [11]:
NN = 20

times = zeros(NN)
map2 = zeros(NN)
distort = zeros(NN)
map2opt = zeros(NN)
distortopt = zeros(NN)
j = 1

1

In [ ]:
for j = 1:20
    times[j] = @elapsed G2,W2 = TreeRep.metric_to_structure(D,undef,undef);
    
    flush(stdout)
    B = W2[1:nv(G2),1:nv(G2)];
    B = sparse(B);
    B = (B .> 0) .* B;
    
    D2 = utilities.parallel_dp_shortest_paths(G2, B,false);
    α = tr(D2[1:n,1:n]'*D)/tr(D2[1:n,1:n]'*D2[1:n,1:n])
    
    map2[j] = utilities.MAP(D2[1:n,1:n],g)
    distort[j] = utilities.avg_distortion(D2[1:n,1:n]*α,D)
    D2 = 0
    B = 0
    GC.gc()
    
    N = size(D)[1]
    
    @show(Sys.free_memory()/2^(30))
    
    L = Int((N*(N-1))/2)
    IDXs = Array{Tuple{Int,Int},1}(undef,L)
    c = 1
    for i = 1:N 
        for j = 1:i-1
            IDXs[c] = (i,j)
            c += 1
        end
    end
    
    @show(Sys.free_memory()/2^(30))
    
    times[j] += @elapsed x,EdgetoIdx = TreeOpt.lsngd_mengdi(G2,D,W2,IDXs,0.0001,200);
    
    IDXs = 0
    W2 = 0
    GC.gc()
    
    N = nv(G2)
    W3 = zeros(N,N)
    E = collect(edges(G2))
    for e in E
        i2 = e.src
        j2 = e.dst
        idx = EdgetoIdx[(i2,j2)]
        w = max(0,x[idx])
        W3[i2,j2] = w
        W3[j2,i2] = w
    end
    
    B = W3[1:nv(G2),1:nv(G2)];
    B = sparse(B);
    B = (B .> 0) .* B;
    D3 = utilities.parallel_dp_shortest_paths(G2, B,false) .+ 1e-14;
    for i = 1:n
        D3[i,i] = 0
    end

    α = tr(D3[1:n,1:n]'*D)/tr(D3[1:n,1:n]'*D3[1:n,1:n])

    distortopt[j] = utilities.avg_distortion(D3[1:n,1:n]*α,D)
    map2opt[j] = utilities.MAP(D3[1:n,1:n],g)
    @show((times[j],distort[j],map2[j]))
end

t = mean(times)
dis = minimum(distort)
m = maximum(map2)
disopt = minimum(distortopt)
mopt = maximum(map2opt)

mdis = mean(distort)
mm = mean(map2)
mdisopt = mean(distortopt)
mmopt = mean(map2opt)

@show((t,dis,m))
@show((t,disopt,mopt))
@show((t,mdis,mm))
@show((t,mdisopt,mmopt))

(nv(G), ne(G)) = (5536, 5535)
Sys.free_memory() / 2 ^ 30 = 9.115985870361328
Sys.free_memory() / 2 ^ 30 = 8.28631591796875


Progress:  98%|████████████████████████████████████████ |  ETA: 0:00:00

Sys.free_memory() / 2 ^ 30 = 0.14223861694335938
Sys.free_memory() / 2 ^ 30 = 0.15359115600585938
(times[j], distort[j], map2[j]) = (80.291470019, 0.19110500291949384, 0.686652532276688)
(nv(G), ne(G)) = (5539, 5538)
Sys.free_memory() / 2 ^ 30 = 8.908164978027344
Sys.free_memory() / 2 ^ 30 = 8.079017639160156


Progress:  97%|████████████████████████████████████████ |  ETA: 0:00:00

In [ ]:
map2

In [ ]:
MAP(D2[1:n,1:n],g)

In [ ]:
avg_distortion(D2[1:n,1:n],D)

In [ ]:
n = nv(g)
map = 0
p2 = Progress(n)
jj = 0
for i = 1:n
    N = neighbors(g,i)
    Dp = D2[1:n,i] .+ 1e-13
    Dp[i] = 0
    p = sortperm(Dp)
    P = Dict()
    for j = 1:n
        P[p[j]] = j
    end
    d = length(N)
    for j = 1:d
        R = P[N[j]]-1
        a = Set(N)
        b = Set(p[2:R+1])
        map += length(intersect(a,b))/(d*R)
    end
    update!(p2,i)
end
    
map/n

Progress:  97%|███████████████████████████████████████▊ |  ETA: 0:01:39

In [15]:
map/n

NaN

In [12]:
d = 0
n = nv(g)
p2 = Progress(n)
for i = 1:n
    for j = 1:i-1
        d += abs(D2[i,j]-D[i,j])/D[i,j]
    end
    update!(p2,i)
end
    
2*d/(n*(n-1))

Progress: 100%|█████████████████████████████████████████| Time: 0:34:43


0.13075636618096464

In [14]:
2*d/(n*(n-1))

0.13075636618096464

In [61]:
NN = 20

times = zeros(NN)
map2 = zeros(NN)
distort = zeros(NN)

for j = 1:NN
    times[j] = @elapsed G2,W2 = TreeRep.metric_to_structure(D,undef,undef);
    
    flush(stdout)
    B = W2[1:nv(G2),1:nv(G2)];
    B = sparse(B);
    B = (B .> 0) .* B;
    
    D2 = utilities.parallel_dp_shortest_paths(G2, B,false);
    
    distort[j] = utilities.avg_distortion(D2[1:n,1:n],D)
    map2[j] = utilities.MAP(D2[1:n,1:n],g)
    
    B = 0
    D2 = 0
    
    GC.gc()
    
    @show((times[j],distort[j],map2[j]))
    flush(stdout)
    
    times[j] += @elapsed A,b,EdgetoIdx,x0 = TreeOpt.makeAbMatrix(G2,D,W2)
    times[j] += @elapsed x,loss = TreeOpt.lsngd(A,b,0.00000001,x0,5000)
    flush(stdout)
    N = nv(G2)
    W2  = zeros(N,N)
    E = collect(edges(G2))
    for e in E
        i2 = e.src
        j2 = e.dst
        idx = EdgetoIdx[(i2,j2)]
        w = max(0,x[idx])
        W2[i2,j2] = w
        W2[j2,i2] = w
    end
    flush(stdout)
    B = W2[1:nv(G2),1:nv(G2)];
    B = sparse(B);
    B = (B .> 0) .* B;
    D2 = utilities.parallel_dp_shortest_paths(G2, B) .+1e-13;
    for i = 1:size(D2)[1]
        D2[i,i] = 0
    end
    flush(stdout)

    distort[j] = utilities.avg_distortion(D2[1:n,1:n],D)
    map2[j] = utilities.MAP(D2[1:n,1:n],g)
    flush(stdout)
    @show((times[j],distort[j],map2[j]))
    flush(stdout)
end

t = mean(times)
dis = minimum(distort)
m = maximum(map2)

@show((t,dis,m))

(nv(G), ne(G)) = (1647, 1646)
(times[j], distort[j], map2[j]) = (2.053427624, 0.39796283945060795, 0.8350589752580877)
 49.618066 seconds (2 allocations: 20.671 MiB)
  0.583949 seconds (1 allocation: 13.000 KiB)
(times[j], distort[j], map2[j]) = (69.615465473, 0.14271864007979382, 0.8056032842023519)
(nv(G), ne(G)) = (1649, 1648)
(times[j], distort[j], map2[j]) = (2.281534681, 0.32903081848989024, 0.8258885414633931)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress:  20%|████████▏                                |  ETA: 0:00:02

 49.041450 seconds (2 allocations: 20.721 MiB)
  0.586563 seconds (1 allocation: 13.000 KiB)
(times[j], distort[j], map2[j]) = (66.176770683, 0.14329930633487334, 0.781786912835245)
(nv(G), ne(G)) = (1644, 1643)
(times[j], distort[j], map2[j]) = (0.43698634, 0.2679632857850257, 0.7982465868309743)
 51.444509 seconds (2 allocations: 20.595 MiB)
  0.579527 seconds (1 allocation: 13.000 KiB)
(times[j], distort[j], map2[j]) = (66.556838933, 0.1389662293875319, 0.7723252527222463)
(nv(G), ne(G)) = (1663, 1662)
(times[j], distort[j], map2[j]) = (0.927457343, 0.5041143197964135, 0.8196828374978425)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress:  28%|███████████▌                             |  ETA: 0:00:00

 50.114799 seconds (2 allocations: 21.074 MiB)
  0.586673 seconds (1 allocation: 13.125 KiB)
(times[j], distort[j], map2[j]) = (66.466327963, 0.13547307852446064, 0.7803161967358179)
(nv(G), ne(G)) = (1740, 1739)
(times[j], distort[j], map2[j]) = (5.054603946, 0.5324882603399064, 0.6843693447329624)


...

 52.479862 seconds (2 allocations: 20.645 MiB)
  0.581556 seconds (1 allocation: 13.000 KiB)
(times[j], distort[j], map2[j]) = (68.45216691, 0.14226392777816368, 0.7953151900923088)
(nv(G), ne(G)) = (1754, 1753)
(times[j], distort[j], map2[j]) = (2.23208785, 0.36550742938216485, 0.6413761147349041)
 56.266463 seconds (2 allocations: 23.445 MiB)
  0.616165 seconds (1 allocation: 13.875 KiB)
(times[j], distort[j], map2[j]) = (75.454326607, 0.14708974850036938, 0.5992126044449996)
(nv(G), ne(G)) = (1662, 1661)
(times[j], distort[j], map2[j]) = (0.615143343, 0.3154074403021912, 0.8286524887113054)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress:  27%|███████████▎                             |  ETA: 0:00:00

 50.612809 seconds (2 allocations: 21.049 MiB)
  0.592096 seconds (1 allocation: 13.125 KiB)
(times[j], distort[j], map2[j]) = (66.695791869, 0.13831661231373577, 0.7996929169280639)
(t, dis, m) = (68.3804516225, 0.13547307852446064, 0.807557887346646)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress:  82%|█████████████████████████████████▌       |  ETA: 0:00:00

(68.3804516225, 0.13547307852446064, 0.807557887346646)

In [15]:
N = 20

times = zeros(N)
map2 = zeros(N)
distort = zeros(N)
D2 = 0
j=1
for j = 1:20
    times[j] = @elapsed G2,W2 = TreeRep.metric_to_structure(D,undef,undef);
    G2 = utilities.remove_loops(G2)
    @show(times[j])
    flush(stdout)
    #@show(is_connected(G2),nv(G2),ne(G2));
    B = W2[1:nv(G2),1:nv(G2)];
    B = sparse(B);
    B = (B .> 0) .* B;
    
    W2 = 0
    GC.gc()
    D2 = utilities.parallel_dp_shortest_paths(G2, B);
    α = trm(D2[1:n,1:n]',D)/(sum(D2.^2))
    distort[j] = utilities.avg_distortion(D2[1:n,1:n]*α,D)
    #map2[j] = utilities.MAP(D2[1:n,1:n],g)
    
    @show((distort[j],map2[j]))
end

(nv(G), ne(G)) = (5570, 5569)
times[j] = 0.53459211


Progress: 100%|████████████████████████████████████████▉|  ETA: 0:00:00

(distort[j], map2[j]) = (0.3776559373835731, 0.0)
(nv(G), ne(G)) = (5469, 5468)
times[j] = 0.685732594


Progress:  40%|████████████████▌                        |  ETA: 0:00:24

InterruptException: InterruptException:

In [13]:
t = mean(times)
dis = maximum(distort)
m = maximum(map2)
mdis = mean(distort)
mm = mean(map2)

@show((t,dis,m))
@show((t,mdis,mm))

(t, dis, m) = (0.51476216335, 0.3995939814022638, 0.0)
(t, mdis, mm) = (0.51476216335, 0.2650421309333974, 0.0)


(0.51476216335, 0.2650421309333974, 0.0)

# Neighbor Join

In [103]:
@time R = NJ.nj!(copy(convert(Matrix{Float64},D)));

  0.793835 seconds (1.04 M allocations: 45.943 MiB, 3.54% gc time)


In [104]:
n

100

In [105]:
m = mean(D)

66.75

In [106]:
g2 = SimpleGraph(R.numNodes)
w = spzeros(R.numNodes,R.numNodes)
for i = 1:R.numEdges
    src = R.edge[i].node[1].number
    dst = R.edge[i].node[2].number
    add_edge!(g2,src,dst)
    w[src,dst] = R.edge[i].length
    w[dst,src] = w[src,dst]
end

In [107]:
@time D5 = utilities.parallel_dp_shortest_paths(g2, w)

  0.003281 seconds (906 allocations: 73.771 KiB)


14×14 Array{Float64,2}:
   0.0      33.875    46.125    49.8125  …   25.25     28.6875   30.25  
  33.875     0.0      26.0      33.1875       8.625    12.0625   13.625 
  46.125    26.0       0.0      45.4375      20.875    24.3125   25.875 
  49.8125   33.1875   45.4375    0.0         24.5625   21.125    19.5625
  48.85     32.225    44.475    41.2875      23.6      20.1625   21.725 
  48.15     31.525    43.775    40.5875  …   22.9      19.4625   21.025 
  97.7708   81.1458   93.3958   87.0833      72.5208   69.0833   67.5208
 151.604   134.979   147.229   140.917      126.354   122.917   121.354 
  50.6875   34.0625   46.3125   40.0         25.4375   22.0      20.4375
  36.5      19.875    32.125    28.9375      11.25      7.8125    9.375 
  27.0       6.875    19.125    26.3125  …    1.75      5.1875    6.75  
  25.25      8.625    20.875    24.5625       0.0       3.4375    5.0   
  28.6875   12.0625   24.3125   21.125        3.4375    0.0       1.5625
  30.25     13.625    25.87

In [74]:
α = tr(D5[1:n,1:n]'*D)/tr(D5[1:n,1:n]'*D5[1:n,1:n])
@show(utilities.avg_distortion(D5[1:n,1:n]*α,D));

utilities.avg_distortion(D5[1:n, 1:n] * α, D) = 0.14237708379400482


In [87]:
@show(utilities.MAP(D5[1:n,1:n],g));

utilities.MAP(D5[1:n, 1:n], g) = 0.5583433628949345


# Making CSV

In [ ]:
n = 1000
D = utilities.rand_hyperbolic(1000,10,1000);

In [ ]:
g = CompleteGraph(n)
E = ne(g)

In [ ]:
id1 = convert(Array{Int64,1},zeros(E))
id2 = convert(Array{Int64,1},zeros(E))
weight = zeros(E)

Ed = collect(edges(g))

for i = 1:E
    e = Ed[i]
    id1[i] = e.src
    id2[i] = e.dst
    weight[i] = D[e.src,e.dst]
end

In [ ]:
df = DataFrame(id1 = id1, id2 = id2, weight = weight)
CSV.write("rand1000.csv",  df, writeheader=true)

In [ ]:
n

In [ ]:
w = adjacency_matrix(r) .* D

# BFS Tree

In [ ]:
g = utilities.kNN(D,10)

In [156]:
G = CompleteGraph(size(D)[1])

{8, 28} undirected simple Int64 graph

In [ ]:
n =nv(g)

In [157]:
@elapsed r = LightGraphs.prim_mst(G,D)

0.122476744

In [158]:
R = SimpleGraph(n)
for e in r
    add_edge!(R,e)
end

In [97]:
@elapsed r = LightGraphs.bfs_tree(g,rand(1:n))

0.004319605

In [98]:
R = SimpleGraph(n)
for e in edges(r)
    add_edge!(R,e)
end

In [12]:
GC.gc()
Sys.free_memory()/2^(30)

13.528114318847656

In [159]:
wbfs =  adjacency_matrix(R) .* D
D3 = utilities.parallel_dp_shortest_paths(R,wbfs);

Progress:  12%|█████▏                                   |  ETA: 0:00:01

In [11]:
function trm(A,B)
    n = size(A)[1]
    t = 0
    for i = 1:n
        t += sum(A[i,:].*B[:,i])
    end
    
    return t
end

trm (generic function with 1 method)

In [15]:
α = trm(D3[1:n,1:n],D)/(sum(D3.^2))

0.7902710903361099

In [16]:
@show(utilities.avg_distortion(D3[1:n,1:n]*α,D));

utilities.avg_distortion(D3[1:n, 1:n] * α, D) = 0.17051780175704664


In [77]:
@show(utilities.MAP(D3[1:n,1:n],g));

utilities.MAP(D3[1:n, 1:n], g) = 0.8691119396937591


In [78]:
N = size(D)[1]
    
@show(Sys.free_memory()/2^(30))
    
L = Int((N*(N-1))/2)
IDXs = Array{Tuple{Int,Int},1}(undef,L)
c = 1
for i = 1:N 
    for j = 1:i-1
        IDXs[c] = (i,j)
        c += 1
    end
end

Sys.free_memory() / 2 ^ 30 = 17.148414611816406


In [79]:
@time x,EdgetoIdx = TreeOpt.lsngd_mengdi(R,D,adjacency_matrix(R),IDXs,0.0001,200);

Sys.free_memory() / 2 ^ 30 = 16.18099594116211
Sys.free_memory() / 2 ^ 30 = 16.18099594116211
  1.988720 seconds (10.33 M allocations: 1.319 GiB)


Progress:  90%|████████████████████████████████████▉    |  ETA: 0:00:00

In [80]:
N = nv(R)
W3 = zeros(N,N)
E = collect(edges(R))
for e in E
    i2 = e.src
    j2 = e.dst
    idx = EdgetoIdx[(i2,j2)]
    w = max(0,x[idx])
    W3[i2,j2] = w
    W3[j2,i2] = w
end

B = W3[1:nv(R),1:nv(R)];
B = sparse(B);
B = (B .> 0) .* B;
D3 = utilities.parallel_dp_shortest_paths(R, B) .+ 1e-14;
for i = 1:n
    D3[i,i] = 0
end


distort = utilities.avg_distortion(D3[1:n,1:n],D)
map2 = utilities.MAP(D3[1:n,1:n],g)

@show((distort,map2))

(distort, map2) = (0.16203463229761678, 0.8583969773161401)


Progress:  91%|█████████████████████████████████████▍   |  ETA: 0:00:00

(0.16203463229761678, 0.8583969773161401)

In [71]:
map = 0
p2 = Progress(n)
for i = 1:n
    N = neighborhood(G,i,1)
    D = D2[:,i]
    p = sortperm(D)
    P = Dict()
    for j = 1:n
        P[p[j]] = j
    end
    d = length(N)
    for j = 1:d
        R = P[N[j]]
        a = Set(N)
        b = Set(p[1:R])
        map += length(intersect(a,b))/(d*R)        
    end
    next!(p2)
end

map/n

UndefVarError: UndefVarError: D2 not defined

# Random Points

In [58]:
N = 10
dist1 = zeros(10)
dist2 = zeros(10)
dist3 = zeros(10)
dist4 = zeros(10)
dist5 = zeros(10)

n = 100

for i = 1:N
    sc = 2^i
    Z = CSV.read("rand-dim10-scale$sc.csv")
    D = zeros(100,100)
    for k = 1:50*99
        ii = Z[k,1]
        j = Z[k,2]
        D[ii,j] = Z[k,3]
        D[j,ii] = Z[k,3]
    end
    n = 100
    #D = utilities.rand_hyperbolic(n,10,2^i)
    
    @time G2, W2 = TreeRep.metric_to_structure(D,undef,undef)
    B = W2[1:nv(G2),1:nv(G2)];
    B = sparse(B);
    B = (B .> 0) .* B;
    D2 = utilities.parallel_dp_shortest_paths(G2, B, false);
    α = tr(D2[1:n,1:n]'*D)/tr(D2[1:n,1:n]'*D2[1:n,1:n])
    dist1[i] = utilities.avg_distortion(D2[1:n,1:n]*α,D)
    
    @time R = NJ.nj!(copy(convert(Matrix{Float64},D)))
    g2 = SimpleGraph(R.numNodes)
    w = spzeros(R.numNodes,R.numNodes)
    for i = 1:R.numEdges
        src = R.edge[i].node[1].number
        dst = R.edge[i].node[2].number
        add_edge!(g2,src,dst)
        w[src,dst] = R.edge[i].length
        w[dst,src] = w[src,dst]
    end
    D5 = utilities.parallel_dp_shortest_paths(g2, w, false);
    dist2[i] = utilities.avg_distortion(D5[1:n,1:n],D)
    
    @time T,W4 = ConstructTree.basicConstructTree(collect(2:n),1,D)
    D4 = LightGraphs.floyd_warshall_shortest_paths(T,W4[1:nv(T),1:nv(T)]).dists;
    α = tr(D4[1:n,1:n]'*D)/tr(D4[1:n,1:n]'*D4[1:n,1:n])
    dist3[i] = utilities.avg_distortion(D4[1:n,1:n]*α,D)
    
    
    #g = CompleteGraph(n)
    g = utilities.kNN(D,10)
    @time r = LightGraphs.prim_mst(g,D)
    #r = LightGraphs.bfs_tree(g,1)
    R = SimpleGraph(n)
    for e in r
        add_edge!(R,e)
    end
    wbfs =  adjacency_matrix(R) .* D
    D3 = utilities.parallel_dp_shortest_paths(R,wbfs);
    α = tr(D3[1:n,1:n]'*D)/tr(D3[1:n,1:n]'*D3[1:n,1:n])
    dist4[i] = utilities.avg_distortion(D3[1:n,1:n]*α,D)
    
    g = CompleteGraph(n)
    #g = utilities.kNN(D,10)
    @time r = LightGraphs.prim_mst(g,D)
    #r = LightGraphs.bfs_tree(g,1)
    R = SimpleGraph(n)
    for e in r
        add_edge!(R,e)
    end
    wbfs =  adjacency_matrix(R) .* D
    D3 = utilities.parallel_dp_shortest_paths(R,wbfs);
    α = tr(D3[1:n,1:n]'*D)/tr(D3[1:n,1:n]'*D3[1:n,1:n])
    dist5[i] = utilities.avg_distortion(D3[1:n,1:n]*α,D)
    
    g = CompleteGraph(n)
    E = ne(g)
    
    id1 = convert(Array{Int64,1},zeros(E))
    id2 = convert(Array{Int64,1},zeros(E))
    weight = zeros(E)

    Ed = collect(edges(g))

    for i = 1:E
        e = Ed[i]
        id1[i] = e.src
        id2[i] = e.dst
        weight[i] = D[e.src,e.dst]
    end
    
    df = DataFrame(id1 = id1, id2 = id2, weight = weight)
    scale = 2^i
    dim = 10
    CSV.write("rand-dim$dim-scale$scale.csv",  df, writeheader=true)
    
    @show((dist1[i],dist2[i]))
end

(nv(G), ne(G)) = (156, 155)
  0.001994 seconds (25.52 k allocations: 1.523 MiB)
  0.018718 seconds (74.45 k allocations: 2.532 MiB)
  0.099853 seconds (627.12 k allocations: 30.673 MiB, 35.39% gc time)
  0.000223 seconds (133 allocations: 20.266 KiB)
  0.000385 seconds (133 allocations: 20.266 KiB)
(dist1[i], dist2[i]) = (0.09036405401168805, 0.05632563045089825)
(nv(G), ne(G)) = (156, 155)
  0.004484 seconds (26.92 k allocations: 1.572 MiB)
  0.028539 seconds (74.45 k allocations: 2.532 MiB, 35.86% gc time)
  0.079803 seconds (627.12 k allocations: 30.672 MiB, 30.11% gc time)
  0.000220 seconds (133 allocations: 20.266 KiB)
  0.000349 seconds (133 allocations: 20.266 KiB)
(dist1[i], dist2[i]) = (0.07084767032069861, 0.04474178277175264)
(nv(G), ne(G)) = (155, 154)
  0.004731 seconds (27.74 k allocations: 1.602 MiB)
  0.019218 seconds (74.45 k allocations: 2.532 MiB)
  0.101166 seconds (627.13 k allocations: 30.720 MiB, 37.97% gc time)
  0.000224 seconds (133 allocations: 20.266 KiB)
 

In [59]:
plot(collect(1:10),dist1,label="TreeRep", lc = :blue, linewidth = 2)
plot!(collect(1:10),dist2,label="NJ", lc = :red, linewidth = 2)
plot!(collect(1:10),dist3,label="ConstructTree", lc = :green, linewidth = 2)
plot!(collect(1:10),dist5,label="MST : Complete", lc = :indigo, linewidth = 2)
plot!(collect(1:10),dist4,label="MST : KNN 10", lc = :darkorange, linewidth = 2)
plot!(collect(1:10),dist6,label="Lorentz Maps", lc = :pink, linewidth = 2)
plot!(collect(1:10),dist8,label="PT", lc = :black, linewidth = 2)
scatter!(collect(1:10),dist1,label="TreeRep", shape=:circle, mc = :blue)
scatter!(collect(1:10),dist2,label="NJ",shape=:cross, mc = :red)
scatter!(collect(1:10),dist3,label="ConstructTree", shape =:xcross, mc = :green)
scatter!(collect(1:10),dist4,label="MST : KNN 10", shape =:vline, mc = :darkorange)
scatter!(collect(1:10),dist5,label="MST : Complete", shape =:diamond, mc = :indigo)
scatter!(collect(1:10),dist6,label="Lorentz Maps", shape = :square, mc = :pink)
scatter!(collect(1:10),dist8,label="PT", shape = :dtriangle, mc = :black)
plot!(xlabel = "Dimension (Log Scale)",ylabel = "Average Distortion (Log scale)",yscale=:log; myfonts...)

In [66]:
N = 10
dist1 = zeros(10)
dist2 = zeros(10)
dist3 = zeros(10)
dist4 = zeros(10)
dist5 = zeros(10)

n = 100

for i = 1:N
    sc = 2^i
    Z = CSV.read("rand-dim$sc-scale1.csv")
    D = zeros(100,100)
    for k = 1:50*99
        ii = Z[k,1]
        j = Z[k,2]
        D[ii,j] = Z[k,3]
        D[j,ii] = Z[k,3]
    end
    n = 100
    #D = utilities.rand_hyperbolic(n,2^i,1)
    
    G2, W2 = TreeRep.metric_to_structure(D,undef,undef)
    B = W2[1:nv(G2),1:nv(G2)];
    B = sparse(B);
    B = (B .> 0) .* B;
    D2 = utilities.parallel_dp_shortest_paths(G2, B, false);
    α = tr(D2[1:n,1:n]'*D)/tr(D2[1:n,1:n]'*D2[1:n,1:n])
    dist1[i] = utilities.avg_distortion(D2[1:n,1:n]*α,D)
    
    R = NJ.nj!(copy(convert(Matrix{Float64},D)))
    g2 = SimpleGraph(R.numNodes)
    w = spzeros(R.numNodes,R.numNodes)
    for i = 1:R.numEdges
        src = R.edge[i].node[1].number
        dst = R.edge[i].node[2].number
        add_edge!(g2,src,dst)
        w[src,dst] = R.edge[i].length
        w[dst,src] = w[src,dst]
    end
    D5 = utilities.parallel_dp_shortest_paths(g2, w, false);
    dist2[i] = utilities.avg_distortion(D5[1:n,1:n],D)
    
    T,W4 = ConstructTree.basicConstructTree(collect(2:n),1,D)
    D4 = LightGraphs.floyd_warshall_shortest_paths(T,W4[1:nv(T),1:nv(T)]).dists;
    α = tr(D4[1:n,1:n]'*D)/tr(D4[1:n,1:n]'*D4[1:n,1:n])
    dist3[i] = utilities.avg_distortion(D4[1:n,1:n]*α,D)
    
    
    #g = CompleteGraph(n)
    g = utilities.kNN(D,10)
    r = LightGraphs.prim_mst(g,D)
    #r = LightGraphs.bfs_tree(g,1)
    R = SimpleGraph(n)
    for e in r
        add_edge!(R,e)
    end
    wbfs =  adjacency_matrix(R) .* D
    D3 = utilities.parallel_dp_shortest_paths(R,wbfs);
    α = tr(D3[1:n,1:n]'*D)/tr(D3[1:n,1:n]'*D3[1:n,1:n])
    dist4[i] = utilities.avg_distortion(D3[1:n,1:n]*α,D)
    
    g = CompleteGraph(n)
    #g = utilities.kNN(D,10)
    r = LightGraphs.prim_mst(g,D)
    #r = LightGraphs.bfs_tree(g,1)
    R = SimpleGraph(n)
    for e in r
        add_edge!(R,e)
    end
    wbfs =  adjacency_matrix(R) .* D
    D3 = utilities.parallel_dp_shortest_paths(R,wbfs);
    α = tr(D3[1:n,1:n]'*D)/tr(D3[1:n,1:n]'*D3[1:n,1:n])
    dist5[i] = utilities.avg_distortion(D3[1:n,1:n]*α,D)
    
    g = CompleteGraph(n)
    E = ne(g)
    
    id1 = convert(Array{Int64,1},zeros(E))
    id2 = convert(Array{Int64,1},zeros(E))
    weight = zeros(E)

    Ed = collect(edges(g))

    for i = 1:E
        e = Ed[i]
        id1[i] = e.src
        id2[i] = e.dst
        weight[i] = D[e.src,e.dst]
    end
    
    df = DataFrame(id1 = id1, id2 = id2, weight = weight)
    scale = 1
    dim = 2^i
    CSV.write("rand-dim$dim-scale$scale.csv",  df, writeheader=true)
    
    @show((dist1[i],dist2[i]))
end

(nv(G), ne(G)) = (108, 107)
(dist1[i], dist2[i]) = (0.3924084905784876, 0.30792713845438285)
(nv(G), ne(G)) = (154, 153)
(dist1[i], dist2[i]) = (0.22702983197222978, 0.15973365814483037)
(nv(G), ne(G)) = (156, 155)
(dist1[i], dist2[i]) = (0.12532154153551622, 0.09197492078873921)
(nv(G), ne(G)) = (151, 150)
(dist1[i], dist2[i]) = (0.07561369994175976, 0.050452637870338206)
(nv(G), ne(G)) = (146, 145)
(dist1[i], dist2[i]) = (0.05047955419678273, 0.03092912658263701)
(nv(G), ne(G)) = (129, 128)
(dist1[i], dist2[i]) = (0.03218813284310934, 0.019087902179392744)
(nv(G), ne(G)) = (116, 115)
(dist1[i], dist2[i]) = (0.019873897316375396, 0.011735821824899339)
(nv(G), ne(G)) = (117, 116)
(dist1[i], dist2[i]) = (0.011902551346127895, 0.007534766866894497)
(nv(G), ne(G)) = (102, 101)
(dist1[i], dist2[i]) = (0.0067557926406194005, 0.004687109709439094)
(nv(G), ne(G)) = (101, 100)
(dist1[i], dist2[i]) = (0.0037224293877966136, 0.0030330320879567284)


In [67]:
plot(collect(1:10),dist1,label="TreeRep", lc = :blue, linewidth = 2)
plot!(collect(1:10),dist2,label="NJ", lc = :red, linewidth = 2)
plot!(collect(1:10),dist3,label="ConstructTree", lc = :green, linewidth = 2)
plot!(collect(1:10),dist5,label="MST : Complete", lc = :indigo, linewidth = 2)
plot!(collect(1:10),dist4,label="MST : KNN 10", lc = :darkorange, linewidth = 2)
plot!(collect(1:10),dist7,label="Lorentz Maps", lc = :pink, linewidth = 2)
plot!(collect(1:10),dist9,label="PT", lc = :black, linewidth = 2)
scatter!(collect(1:10),dist1,label="TreeRep", shape=:circle, mc = :blue)
scatter!(collect(1:10),dist2,label="NJ",shape=:cross, mc = :red)
scatter!(collect(1:10),dist3,label="ConstructTree", shape =:xcross, mc = :green)
scatter!(collect(1:10),dist4,label="MST : KNN 10", shape =:vline, mc = :darkorange)
scatter!(collect(1:10),dist5,label="MST : Complete", shape =:diamond, mc = :indigo)
scatter!(collect(1:10),dist7,label="Lorentz Maps", shape = :square, mc = :pink)
scatter!(collect(1:10),dist9,label="PT", shape = :dtriangle, mc = :black)
plot!(xlabel = "Scale (Log scale)",ylabel = "Average Distortion (Log scale)",yscale=:log; myfonts...)

In [ ]:
dist1

In [16]:
@load "dist6.jld2" dist6
@load "dist7.jld2" dist7

1-element Array{Symbol,1}:
 :dist7

In [17]:
@load "dist8.jld2" dist8

1-element Array{Symbol,1}:
 :dist8

In [18]:
@load "dist9.jld2" dist9

1-element Array{Symbol,1}:
 :dist9

In [107]:
dist8

10-element Array{Float64,1}:
 0.8546943005658045  
 0.6642194310670083  
 0.22416677269118204 
 0.12312041458349997 
 0.11469794716417758 
 0.08772766088122697 
 0.042399332866795626
 0.02946842818590852 
 0.023851548199430625
 0.020019080198004524

In [108]:
dist9

10-element Array{Float64,1}:
 0.8546943005658045  
 0.6642194310670083  
 0.22416677269118204 
 0.12312041458349997 
 0.11469794716417758 
 0.08772766088122697 
 0.042399332866795626
 0.02946842818590852 
 0.023851548199430625
 0.020019080198004524